<a href="https://colab.research.google.com/github/Arghavankas2022/NNwithpytorch/blob/main/NN_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install torch torchvision torchaudio

In [ ]:
!pip3 list -f pytorch

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('pima-indians-diabetes.data.csv', delimiter=',')
X = dataset[:, 0:8]
y = dataset[:, 8]

# Normalize the input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

# Define the model
class PimaClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(8, 12)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(12, 8)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(8, 1)
        self.act_output = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PimaClassifier().to(device)

# Define loss function and optimizer
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 130
batch_size = 13

# Training loop
for epoch in range(n_epochs):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i + batch_size].to(device)
        y_batch = y_train[i:i + batch_size].to(device)

        # Forward pass
        y_pred = model(X_batch)

        # Calculate loss
        loss = loss_fn(y_pred, y_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Evaluate accuracy on the test set
model.eval()
with torch.no_grad():
    y_pred = model(X_test.to(device))
    y_pred = (y_pred > 0.5).float()  # Convert to binary predictions

accuracy = (y_pred == y_test.to(device)).float().mean()
print(f"Accuracy: {accuracy.item()}")

# Make predictions on the first 5 samples
predictions = (model(X_test[:5].to(device)) > 0.5).int()
for i in range(5):
    print('%s => %d (expected %d)' % (X_test[i].tolist(), predictions[i], y_test[i]))


Accuracy: 0.7792207598686218
[0.639947235584259, -0.7165334820747375, -0.574127733707428, 0.7818138003349304, 0.9568596482276917, 0.25478047132492065, -0.1264714002609253, 0.8303811550140381] => 0 (expected 0)
[-0.5479186177253723, -0.27837345004081726, 0.3047339916229248, 0.7190857529640198, -0.6928905844688416, 0.47054317593574524, -0.9781448841094971, -1.0415494441986084] => 0 (expected 0)
[-0.5479186177253723, -0.4035620093345642, -0.2639412581920624, -1.2882121801376343, -0.6928905844688416, -0.1513611227273941, -0.9479436874389648, -1.0415494441986084] => 0 (expected 0)
[1.2338801622390747, -0.43485915660858154, 0.5632227659225464, -1.2882121801376343, -0.6928905844688416, -0.9382604360580444, 1.1600991487503052, 0.06459135562181473] => 0 (expected 0)
[0.9369137287139893, 0.47275805473327637, 1.0802001953125, -1.2882121801376343, -0.6928905844688416, -0.2655884325504303, -0.7908974885940552, 1.4259953498840332] => 0 (expected 0)
